# **Applied Statistics Problems Notebook**

This notebook contains my answers to each of the four problems given in the assessmnet in the Applied Statistics module.

The aim of the notebook is to show an understanding of the principals of statistics and to demonstrate the following abilities, taken from the learning outcomes of the module:

- Describe the stochastic nature of real-world measurements.

- Source documentation to programmatically perform a statistical test.

- Select an appropriate statistical test to investigate a claim.

- Perform a statistical test on a data set.

Within this notebook `Code blocks` are used wherever code is referenced.

***
## Problem 1: Extending the Lady Tasting Tea

In this first

***
## Problem 2: Normal Distribution

For the second task,

***
## Problem 3: t-Tests

For the third task,

***
## Problem 4: ANOVA

In this final task

***